In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.types import DoubleType

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
df = spark.read.csv("WorldCupMatches.csv",header=True);
df.show()
print(df.schema)

+----+--------------------+-----------------+--------------------+-----------+--------------+---------------+---------------+--------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|Year|            Datetime|            Stage|             Stadium|       City|Home Team Name|Home Team Goals|Away Team Goals|Away Team Name|      Win conditions|Attendance|Half-time Home Goals|Half-time Away Goals|             Referee|         Assistant 1|         Assistant 2|RoundID|MatchID|Home Team Initials|Away Team Initials|
+----+--------------------+-----------------+--------------------+-----------+--------------+---------------+---------------+--------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|193

In [4]:
df.columns

['Year',
 'Datetime',
 'Stage',
 'Stadium',
 'City',
 'Home Team Name',
 'Home Team Goals',
 'Away Team Goals',
 'Away Team Name',
 'Win conditions',
 'Attendance',
 'Half-time Home Goals',
 'Half-time Away Goals',
 'Referee',
 'Assistant 1',
 'Assistant 2',
 'RoundID',
 'MatchID',
 'Home Team Initials',
 'Away Team Initials']

In [34]:
df=df.withColumnRenamed('Home Team Name', 'Home_Team')
df=df.withColumnRenamed('Away Team Name', 'Away_Team')
df=df.withColumnRenamed('Away Team Goals', 'Away_Team_Goals')
df=df.withColumnRenamed('Home Team Goals', 'Home_Team_Goals')
df=df.withColumnRenamed('Assistant 1', 'Assistant_1')
df=df.withColumnRenamed('Assistant 2', 'Assistant_2')
df=df.withColumnRenamed('Home Team Initials', 'Home_Team_Initials')
df=df.withColumnRenamed('Away Team Initials', 'Away_Team_Initials')
df=df.withColumnRenamed('Half-time Home Goals', 'Half-time_Home_Goals')
df=df.withColumnRenamed('Half-time Away Goals', 'Half-time_Away_Goals')

In [35]:
df.columns

['Year',
 'Datetime',
 'Stage',
 'Stadium',
 'City',
 'Home_Team',
 'Home_Team_Goals',
 'Away_Team_Goals',
 'Away_Team',
 'Win conditions',
 'Attendance',
 'Half-time_Home_Goals',
 'Half-time_Away_Goals',
 'Referee',
 'Assistant_1',
 'Assistant_2',
 'RoundID',
 'MatchID',
 'Home_Team_Initials',
 'Away_Team_Initials']

In [36]:
df=df.withColumn('Away_Team_Goals',df['Away_Team_Goals'].cast(DoubleType()))
df=df.withColumn('Home_Team_Goals',df['Home_Team_Goals'].cast(DoubleType()))
df=df.withColumn('Attendance',df['Attendance'].cast(DoubleType()))
df=df.withColumn('Home_Team_Goals',df['Home_Team_Goals'].cast(DoubleType()))
df=df.withColumn('Away_Team_Goals',df['Away_Team_Goals'].cast(DoubleType()))

In [37]:
df.describe(['Year']).show()

+-------+------------------+
|summary|              Year|
+-------+------------------+
|  count|               852|
|   mean|1985.0892018779343|
| stddev|22.448824702021138|
|    min|              1930|
|    max|              2014|
+-------+------------------+



In [38]:
df.filter((df.Stage.like("Final") | df.Stage.like("Semi-finals")) & (df.Away_Team == 'Czechoslovakia') & (df.Home_Team == 'Italy')).count()

1

In [39]:
df.filter(df.Year.like("2014") | df.Year.like("2013")).show(10)

+----+--------------------+-------+------------------+---------------+-------------+---------------+---------------+-----------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+---------+------------------+------------------+
|Year|            Datetime|  Stage|           Stadium|           City|    Home_Team|Home_Team_Goals|Away_Team_Goals|  Away_Team|Win conditions|Attendance|Half-time_Home_Goals|Half-time_Away_Goals|             Referee|         Assistant_1|         Assistant_2|RoundID|  MatchID|Home_Team_Initials|Away_Team_Initials|
+----+--------------------+-------+------------------+---------------+-------------+---------------+---------------+-----------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+---------+------------------+------------------+
|2014|12 Jun 2014 - 17:00 |Group A|Arena de Sao Paul

In [40]:
print("Matches in 2014 and 2013 : " + str(df.filter(df.Year.like("2014") | df.Year.like("2013")).count()))

Matches in 2014 and 2013 : 80


In [41]:
df.groupBy('City').count().show(15,truncate=True)

+------------------+-----+
|              City|count|
+------------------+-----+
|            Daegu |    4|
|            Paris |    9|
|            Natal |    4|
|    San Francisco |    6|
|Santiago De Chile |   10|
|       Eskilstuna |    1|
|        La Coru�A |    3|
|           Bilbao |    3|
|           Geneva |    4|
|         Le Havre |    1|
|           Verona |    4|
|             Kobe |    3|
|            Solna |    8|
|        Liverpool |    5|
|          Gwangju |    3|
+------------------+-----+
only showing top 15 rows



In [44]:
df.groupBy('Home_Team').max('Home_Team_Goals').dropna().show()

+---------+--------------------+
|Home_Team|max(Home_Team_Goals)|
+---------+--------------------+
| Paraguay|                 3.0|
|   Russia|                 6.0|
|  Senegal|                 3.0|
|   Sweden|                 8.0|
|  IR Iran|                 0.0|
|   Turkey|                 7.0|
|    Zaire|                 0.0|
|     Iraq|                 1.0|
|  Germany|                 8.0|
|   France|                 7.0|
|   Greece|                 2.0|
|  Algeria|                 3.0|
|     Togo|                 0.0|
| Slovakia|                 3.0|
|Argentina|                 6.0|
|    Wales|                 2.0|
|  Belgium|                 3.0|
|   Angola|                 0.0|
|  Ecuador|                 3.0|
|    Ghana|                 2.0|
+---------+--------------------+
only showing top 20 rows



In [51]:
df.groupBy('Away_Team').sum('Away_Team_Goals').orderBy('Away_Team').dropna().show(15,truncate=True)

+--------------------+--------------------+
|           Away_Team|sum(Away_Team_Goals)|
+--------------------+--------------------+
|"rn"">Bosnia and ...|                 1.0|
|"rn"">Republic of...|                 8.0|
|"rn"">Serbia and ...|                 2.0|
|"rn"">Trinidad an...|                 0.0|
|"rn"">United Arab...|                 2.0|
|             Algeria|                 9.0|
|              Angola|                 1.0|
|           Argentina|                22.0|
|           Australia|                 4.0|
|             Austria|                12.0|
|             Belgium|                27.0|
|             Bolivia|                 0.0|
|              Brazil|                45.0|
|            Bulgaria|                11.0|
|            Cameroon|                 7.0|
+--------------------+--------------------+
only showing top 15 rows



In [49]:
from pyspark.sql.functions import col, struct

df.withColumn("Home_Team_By_Year", struct("Year", "Home_Team")).crosstab("Home_Team_By_Year", "Stage")

DataFrame[Home_Team_By_Year_Stage: string, Final: bigint, First round: bigint, Group 1: bigint, Group 2: bigint, Group 3: bigint, Group 4: bigint, Group 5: bigint, Group 6: bigint, Group A: bigint, Group B: bigint, Group C: bigint, Group D: bigint, Group E: bigint, Group F: bigint, Group G: bigint, Group H: bigint, Match for third place: bigint, Play-off for third place: bigint, Preliminary round: bigint, Quarter-finals: bigint, Round of 16: bigint, Semi-finals: bigint, Third place: bigint, null: bigint]

In [54]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

lines = sc.textFile("WorldCups.csv", 1)
header = lines.first()
content = lines.filter(lambda line: line != header)

rdd = content.map(lambda line: (line.split(","))).collect()

rdd_len = content.map(lambda line: len(line.split(","))).distinct().collect()

print(rdd_len)

[10]


In [56]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType,IntegerType

schema = StructType([StructField('Year', StringType(), True),
StructField('Country', StringType(),True),
StructField('Winner', StringType(), True),
StructField('Runners-Up', StringType(),True),
StructField('Third', StringType(),True),
StructField('Fourth', StringType(),True),
StructField('GoalsScored', StringType(),True),
StructField('QualifiedTeams', StringType(),True),
StructField('MatchesPlayed', StringType(),True),
StructField('Attendance', StringType(),True)])

In [66]:
print("---------- 1. Venues & goals scored ------------")

# venue - hosted country with highest goals (From RDD)
rdd1 = (content.filter(lambda line: line.split(",")[6] != "NULL")
.map(lambda line: (line.split(",")[1], int(line.split(",")[6])))
.takeOrdered(10, lambda x : -x[1]))
print(rdd1)


# Create data frame from the RDD
df = spark.createDataFrame(rdd,schema)

df=df.withColumn('GoalsScored',df['GoalsScored'].cast(IntegerType()))

# venue - hosted country with highest goals (From DF)
df.select("Country","GoalsScored").orderBy("GoalsScored", ascending = False).show(20, truncate = False)

# venue - hosted country with highest goals (From DF - SQL)
df.createOrReplaceTempView("df_table")
spark.sql(" SELECT Country,GoalsScored FROM df_table order by "+ 
"GoalsScored Desc Limit 10").show()

---------- 1. Venues & goals scored ------------
[('France', 171), ('Brazil', 171), ('Korea/Japan', 161), ('Germany', 147), ('Spain', 146), ('South Africa', 145), ('USA', 141), ('Switzerland', 140), ('Mexico', 132), ('Sweden', 126)]
+------------+-----------+
|Country     |GoalsScored|
+------------+-----------+
|France      |171        |
|Brazil      |171        |
|Korea/Japan |161        |
|Germany     |147        |
|Spain       |146        |
|South Africa|145        |
|USA         |141        |
|Switzerland |140        |
|Mexico      |132        |
|Sweden      |126        |
|Italy       |115        |
|Argentina   |102        |
|Germany     |97         |
|Mexico      |95         |
|England     |89         |
|Chile       |89         |
|Brazil      |88         |
|France      |84         |
|Uruguay     |70         |
|Italy       |70         |
+------------+-----------+

+------------+-----------+
|     Country|GoalsScored|
+------------+-----------+
|      France|        171|
|      Bra

In [73]:
print("-------------- 2. Details of years starting in 2010 ---------------")

# using RDD
years = ["2010", "2011", "2012", "2013", "2014"]
(content.filter(lambda line: line.split(",")[0] in years)
.map(lambda line: (line.split(",")[0],line.split(",")[2],line.split(",")[4])).collect())
    
# using DF
df.select("Year","Winner","Third").filter(df.Year.isin(years)).show()

# using DF - SQL    
spark.sql(" SELECT Year,Winner,Third FROM df_table  WHERE " + " Year IN ('2010','2011','2012','2013','2014') ").show()

-------------- 3. Details of years starting in 2010 ---------------
+----+-------+-----------+
|Year| Winner|      Third|
+----+-------+-----------+
|2010|  Spain|    Germany|
|2014|Germany|Netherlands|
+----+-------+-----------+

+----+-------+-----------+
|Year| Winner|      Third|
+----+-------+-----------+
|2010|  Spain|    Germany|
|2014|Germany|Netherlands|
+----+-------+-----------+



In [70]:
print("---------- 3. Year, venue country = Third Place ------------")

# using RDD
(content.filter(lambda line:line.split(",")[1]==line.split(",")[4])
 .map(lambda line: (line.split(",")[0],line.split(",")[1], line.split(",")[4]))
 .collect())
    
# using DF
df.select("Year","Country","Third").filter(df["Country"]==df["Third"]).show()

# using DF - SQL
spark.sql(" SELECT Year,Country,Third FROM df_table where Country == Third order by Year").show()

---------- 2. Year, venue country = Third Place ------------
+----+-------+-------+
|Year|Country|  Third|
+----+-------+-------+
|1962|  Chile|  Chile|
|1990|  Italy|  Italy|
|2006|Germany|Germany|
+----+-------+-------+

+----+-------+-------+
|Year|Country|  Third|
+----+-------+-------+
|1962|  Chile|  Chile|
|1990|  Italy|  Italy|
|2006|Germany|Germany|
+----+-------+-------+



In [76]:
print("------------- 4. Max Goals Scored -----------------")

# Using RDD
(content.filter(lambda line:line.split(",")[6] == "171")
.map(lambda line: (line.split(","))).collect())

# using DF
df=df.withColumn('GoalsScored',df['GoalsScored'].cast(IntegerType()))
df.filter(df.GoalsScored == 171).show()

# using DF - SQL
spark.sql(" Select * from df_table where GoalsScored in " +
"(Select Max(GoalsScored) from df_table )" ).show()

------------- 4. Max matches played -----------------
+----+-------+-------+----------+-----------+-----------+-----------+--------------+-------------+----------+
|Year|Country| Winner|Runners-Up|      Third|     Fourth|GoalsScored|QualifiedTeams|MatchesPlayed|Attendance|
+----+-------+-------+----------+-----------+-----------+-----------+--------------+-------------+----------+
|1998| France| France|    Brazil|    Croatia|Netherlands|        171|            32|           64| 2.785.100|
|2014| Brazil|Germany| Argentina|Netherlands|     Brazil|        171|            32|           64| 3.386.810|
+----+-------+-------+----------+-----------+-----------+-----------+--------------+-------------+----------+

+----+-------+-------+----------+-----------+-----------+-----------+--------------+-------------+----------+
|Year|Country| Winner|Runners-Up|      Third|     Fourth|GoalsScored|QualifiedTeams|MatchesPlayed|Attendance|
+----+-------+-------+----------+-----------+-----------+--------

In [78]:
print("-------------- 4. 2010 world cup stats --------------")
# using RDD
(content.filter(lambda line:line.split(",")[0]=="2010")
.map(lambda line: (line.split(","))).collect())
# using DF
df.filter(df.Year=="2010").show()
# using DF - Sql
spark.sql(" Select * from df_table where Year == 2010 ").show()

-------------- 4. 2010 world cup stats --------------
+----+------------+------+-----------+-------+-------+-----------+--------------+-------------+----------+
|Year|     Country|Winner| Runners-Up|  Third| Fourth|GoalsScored|QualifiedTeams|MatchesPlayed|Attendance|
+----+------------+------+-----------+-------+-------+-----------+--------------+-------------+----------+
|2010|South Africa| Spain|Netherlands|Germany|Uruguay|        145|            32|           64| 3.178.856|
+----+------------+------+-----------+-------+-------+-----------+--------------+-------------+----------+

+----+------------+------+-----------+-------+-------+-----------+--------------+-------------+----------+
|Year|     Country|Winner| Runners-Up|  Third| Fourth|GoalsScored|QualifiedTeams|MatchesPlayed|Attendance|
+----+------------+------+-----------+-------+-------+-----------+--------------+-------------+----------+
|2010|South Africa| Spain|Netherlands|Germany|Uruguay|        145|            32|        